# Exploratory Data Analysis

---

#### Author: Damian Pietroń & Dominik Cedro

---

## Imports

- Access the directory containing the CSV files.
- Store the paths of all CSV files in the `all_csv` variable, which is of type `list[string]`.

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import datetime
import xlrd
from tabulate import tabulate
import seaborn as sns
# import neurokit2 as nk 
import os
from statsmodels.tsa.stattools import acf, q_stat
from statsmodels.tsa.seasonal import seasonal_decompose

from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler

In [4]:
data_folder = Path("data_clean") 
all_csv = list(data_folder.glob("*.csv"))

---

## Preprocessing

- Initialize the `data_frames` dictionary to hold the DataFrame for each patient's data.
- Encode `day` & `hour` & `time` columns
- Set `DateTime` as an index of data
- Note: DataTime column was encoded into `pd.DataTime` when df was cleaned

`data_frames` structure: 

| Key        | Type   | Value          | Type              |
|------------|--------|----------------|-------------------|
| `file_name`| string | `patient_data` | `pd.DataFrame`    |

In [5]:
def time_encoder(df):
  """
  Encode time features from datetime column
  
  Args:
      df (DataFrame): Dataframe to encode time features
      
  Returns:
      df (DataFrame): Dataframe with encoded time features
  """
  df['time'] = pd.to_datetime(df['DateTime'])
  df['hour'] = df['time'].dt.hour
  df['day'] = df['time'].dt.day
  return df

In [6]:
data_frames = {}
for file in tqdm(all_csv):
  df = pd.read_csv(file, sep=",", decimal=".")
  time_encoder(df)
  df.set_index('DateTime', inplace=True)
  data_frames[file.stem] = df

100%|██████████| 5/5 [00:00<00:00, 17.92it/s]


---

## Data Understanding

- print basic descryption of data 
  - we can see that mean of std threw dataframes is around  8 [a.u]
  - This is vital inforamtion because we would be using `neg_mean_sqaure_error` for models so in order to see how our model performs we should compare its score to this std 
- plot pairplot to see how data is distributed 
  - here we can see that `day` is not highly distibuted and we may want to exclude it from features
  - also we can see that `hour` variable is highly distibuted so it might have strong correlation between `ABP` variable
- We have massive amount of features so we might not want to engineer them futurer beacuse it would massively increase dimensionality of data - this is my hypothesis **Damian Pietroń** 

In [7]:
for key, df in data_frames.items():
    print(f"DataFrame: {key}")
    print(tabulate(df.describe(), headers='keys', tablefmt='pretty'))
    print("\n")  

DataFrame: PAC06_cleaned
+-------+-------------------+--------------------+--------------------+--------------------+---------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------------+-------------------------------+-------------------+--------------------+
|       |        ABP        |   ABP_BaroIndex    | ABP_HRVstats_RMSSD | ABP_HRVstats_SDSD  |    ABP_HRVpsd_LF    |   ABP_HRVpsd_HF    | ABP_HRVpsd_LF_to_HF |   ABP_HRVpsd_TP    |    ABP_FundAmp     |         HR         |        ICP        |       ETCO2        |         Prx         |             time              |       hour        |        day         |
+-------+-------------------+--------------------+--------------------+--------------------+---------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------------------+--------------------+

In [ ]:
for patient in data_frames:
  sns.pairplot(data_frames[patient], diag_kind='kde', plot_kws={'alpha':0.5, 's':80, 'edgecolor':'k'}, height=2)

C:\Users\Domin\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\Domin\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\Domin\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\Domin\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating

---

## Correlation

- Compute the correlation among dataset variables to identify relationships.
- Visualize these correlations using a heatmap, aiding in the selection of variables beneficial for model training.
- Capture the correlation metrics in the `corr_values` variable, designated as a `list`.
- Transform the correlation data to facilitate comparison of ABP-related correlations among different patients.
- Employ a PairGrid plot to illustrate the comparison of correlation metrics across patients.

In [ ]:
def corr_melt(corr):
    """
    Melt correlation matrix to be used to compare correlation between patients
    
    Args:
        df (DataFrame): Dataframe to melt
        
    Returns:
        df_melt (DataFrame): Melted dataframe
    """
    abp_corr = corr['ABP'].reset_index()    
    corr_melted = abp_corr.melt(id_vars = ['ABP'])
    corr_melted['patient'] = key
    return corr_melted

In [ ]:
corr_values = []
for key, df in tqdm(data_frames.items()):
    data_frames[key] = df.dropna()
    plt.figure(figsize=(20, 20)) 
    corr = data_frames[key].corr()
    corr_values.append(corr_melt(corr))
    sns.heatmap(corr, annot=True, cmap='coolwarm', square=True)
    plt.show()

### Correlation between patients

The analysis reveals that `FundAmp` demonstrates the highest and most consistent correlation among the columns. This observation suggests that features in the **frequency domain** could be particularly valuable for model training.

In [ ]:
# Concatenate all correlation values into a single DataFrame for visualization
corr_values_df = pd.concat(corr_values)

# Set the visual theme for the seaborn plots
sns.set_theme(style="whitegrid")

# Initialize a PairGrid object for plotting, specifying the DataFrame, variables, and aesthetics
g = sns.PairGrid(corr_values_df, y_vars='value', x_vars='ABP', hue='patient', height=10, aspect=2)

# Map a stripplot to the PairGrid, configuring its appearance and behavior
g.map(sns.stripplot, size=10, orient="h", jitter=False,
      palette="flare_r", linewidth=1, edgecolor="w")

# Set the labels for the x and y axes
g.set(xlabel='Correlation', ylabel='Values')

# Add a legend to the plot to identify each patient by color
g.add_legend()

# Iterate over each subplot in the PairGrid to set individual titles and enable grid lines on the y-axis
for ax, title in zip(g.axes.flat, data_frames.keys()):
    ax.set(title=title)  # Set the title for each subplot based on patient data
    ax.yaxis.grid(True)  # Enable grid lines on the y-axis for better readability

# Remove the top and right spines from the plot for a cleaner look
sns.despine(left=True, bottom=True)

# Display the plot
plt.show()

Based on this preliminary analysis, we can conclude that `ABP_Fund_Amp` exhibits  strongest and most stable correlation with ABP recordings.

---

## Data Seasonality

- perform Seasonal decomposition of Data



In [ ]:
# D - checking the sizes of data that are being put into seasonal decomposition
for patient, df in data_frames.items():
    print(f"Data for patient: {patient}")
    df.info()
    print("\n")

In [ ]:
for patient in data_frames.keys():

    subset_size = 1000
    subset = data_frames[patient].iloc[:subset_size]
    result = seasonal_decompose(subset['ABP'], period=60, model='additive')
    result.plot()
    plt.show()

Reduced sample allowed for quicker computation. Period 60 was chosen based on hourly seasonal changes (data is recorded every minute).

In [ ]:
for patient in data_frames.keys():
    data_frames[patient]['ABP'] = pd.to_numeric(data_frames[patient]['ABP'])
    result = seasonal_decompose(data_frames[patient]['ABP'], period=60, model='additive')
    result.plot()
    plt.show()

---

## Autocorrelation

- Perform autocorelation study on dataset 
- Data seeems to exhibts high autocorrelation
  
    

In [ ]:
for patient in data_frames.keys():
  acf_values = acf(data_frames[patient]['ABP'], nlags=100)
  q_values, p_values = q_stat(acf_values[1:], len(data_frames[patient]['ABP']))
  print('P-values from Ljung-Box Test:', p_values)

After rejecting null-hypothesis next step is to show the ACF/PACF plots. This will help acknowledge the autocorrelation better. First I will check for stationarity.

In [ ]:
from statsmodels.tsa.stattools import adfuller
for patient in data_frames.keys():
    
    result = adfuller(data_frames[patient]['ABP'])
    
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

When stationarity confirmed I continue to plot acf and pacf.

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

for patient in data_frames.keys():

    plt.figure(figsize=(12,4))
    plot_acf(data_frames[patient]['ABP'], lags=100, ax=plt.gca())
    plt.title('Autocorrelation plot of ABP')
    plt.show()
    
    plt.figure(figsize=(12,4))
    plot_pacf(data_frames[patient]['ABP'], lags=100, ax=plt.gca())
    plt.title('Partial Autocorrelation plot of ABP')
    plt.show()

ACF and PACF look consistent across all patients datasets. There is a very strong autocorrelation out of confidence interval throughout whole 100 lags and beyond. Autocorrelation slowly fades.

---

## Feature Engineering

### Additional descriptors 

#### Neurokit - Frequency Features

To derive frequency domain HRV (Heart Rate Variability) features, it was initially believed that analyzing 1-hour long recordings would be beneficial. However, this approach proved to be misleading. The dataset comprises numerous HRV recordings, which are aggregated by a device into one-minute average samples. Due to this aggregation, accessing specific HRV features is impractical because the sampling frequency is too low to support detailed frequency domain analysis.

#### close to 1 hour long non medical features

Altho we cannot acces Neurokit Features it might be beneficial to own one hour long recordings and calculate features based on them
<span style="color: red;">- dominik</span> 

- Calculate time_diffs.
- Calculate avg time diffs in seconds.
- Create windows based on avg time diffs and selected window size.
- Store those windows in `windows_from_all_patients` variable.
- plot data of a patient sliced to 1 hour windows.

`windows_from_all_patients` structure:
| Key        | Type   | Value          | Type              |List[Type] |
|------------|--------|----------------|-------------------|-----------|
| `file_name`| string |    `windows`    |       `list`      | `pd.DataFrame`|

In [ ]:
for patient in data_frames.keys():
  data_frames[patient].reset_index(inplace=True)
  
  time_diffs = data_frames[patient]['time'].diff().dropna()
  time_diffs_in_seconds = time_diffs.dt.total_seconds()
    
  average_sampling_interval = time_diffs_in_seconds.mean()
  print("Average Sampling Interval for Patient", patient, ":", average_sampling_interval, "seconds")
  
  data_frames[patient].set_index('DateTime', inplace=True)

In [ ]:
def create_windows(df, window_size_in_minutes, average_sampling_interval):
    """
    This function creates windows of the given size from the data.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the data.
    window_size_in_minutes : int
        The size of the window in minutes.
    sampling_frequency : float
        The sampling frequency of the data.
    
    Returns
    -------
    windows : pandas.DataFrame
        The windows of the given size.
        
    Notes:
    ------
    
    Window Size Calculation:
    
    The function first converts the window_size_in_minutes 
    into seconds by multiplying it by 60. It then calculates 
    the number of samples that fit into a window of the 
    specified size, given the average_sampling_interval. 
    This is done by dividing the window size in seconds by the 
    avg sampling interval in seconds, resulting in the window size 
    in samples. The result is rounded down to the nearest 
    integer to ensure that only complete samples are included
    in each window.
    
    Window Creation:
    
    The function initializes an empty list, windows, to store 
    the resulting windows. It then iterates over the DataFrame 
    in steps equal to the calculated window size in samples. 
    For each iteration, it selects a slice of the DataFrame 
    corresponding to the current window and appends this slice 
    to the windows list.
    
    """
    windows = []
    window_size_in_seconds = window_size_in_minutes * 60
    
    window_size_in_samples = window_size_in_seconds / average_sampling_interval
    window_size_in_samples = np.floor(window_size_in_samples).astype(int)
    print(f"Window size in samples: {window_size_in_samples}")
    for i in range(0, len(df), window_size_in_samples):
        window = df.iloc[i:i + window_size_in_samples]
        windows.append(window)
    
    return windows  # return only the first window

In [ ]:
windows_from_all_patients = {}
for patient in tqdm(data_frames.keys()):
    all_windows = create_windows(data_frames[patient], 60, average_sampling_interval)
    windows_from_all_patients[patient] = all_windows
    print(f"Number of windows: {len(all_windows)}")

In [ ]:
for patient_data, values in windows_from_all_patients.items():
    plt.figure(figsize=(20, 10))
    for vvalue in values:
        plt.plot(vvalue['ABP'])
    plt.show()

---

## Save data

In [ ]:
if not os.path.exists('data_to_learn'):
    os.makedirs('data_to_learn')

In [ ]:
for patient in data_frames.keys():
    data_frames[patient].to_csv(f"data_to_learn/{patient}_data_to_learn.csv")

---

Feature engineering part. I decided to start of with features based on window statistics.

In [ ]:
import tqdm
def feature_engineering(df):
    """
    Perform feature engineering on a given dataframe.
    
    Args:
        df (DataFrame): DataFrame containing the time series data.
        
    Returns:
        features (DataFrame): DataFrame containing engineered features.
    """
    if len(df) < 60:
        # Return an empty DataFrame if there are not enough 
        return pd.DataFrame()
    
    features = pd.DataFrame(index=df.index)

    features['mean'] = df['ABP'].rolling(window=60).mean()
    features['std'] = df['ABP'].rolling(window=60).std()
    features['min'] = df['ABP'].rolling(window=60).min()
    features['max'] = df['ABP'].rolling(window=60).max()
    features['median'] = df['ABP'].rolling(window=60).median()
    features['range'] = features['max'] - features['min']
    features['skew'] = df['ABP'].rolling(window=60).apply(skew, raw=True)
    features['kurtosis'] = df['ABP'].rolling(window=60).apply(kurtosis, raw=True)
    
    features['rolling_mean'] = df['ABP'].rolling(window=60).mean()
    features['rolling_std'] = df['ABP'].rolling(window=60).std()
    features['expanding_mean'] = df['ABP'].expanding().mean()
    features['expanding_std'] = df['ABP'].expanding().std()
    
    for lag in range(1, 4): 
        features[f'lag_{lag}'] = df['ABP'].shift(lag)
    
    features = features.dropna()
    
    return features

# I will hold the results in dictionaries below
engineered_features = {}
scaled_features = {}

for patient, windows in tqdm(windows_from_all_patients.items(), desc="Processing patients"):
    patient_features = []
    
    for window in windows:
        window_features = feature_engineering(window)
        
        # check in case the window is empty?
        if not window_features.empty:  
            patient_features.append(window_features)
    
    if patient_features:  
        engineered_features[patient] = pd.concat(patient_features)
    else:
        engineered_features[patient] = pd.DataFrame() 


scaler = StandardScaler()

for patient, features in engineered_features.items():
    if not features.empty:
        scaled_features[patient] = scaler.fit_transform(features)
        scaled_features[patient] = pd.DataFrame(scaled_features[patient], index=features.index, columns=features.columns)

if not os.path.exists('engineered_data'):
    os.makedirs('engineered_data')

for patient, features in scaled_features.items():
    if not features.empty: 
        features.to_csv(f"engineered_data/{patient}_engineered.csv")
    else:
        print("no features saved")

print("Feature engineering completed and data saved.")
